# Домашнее задание 3

Весь код архиватора лежит тут: https://github.com/Andrew-Zlobin/CMDC/tree/main/task3

Смотреть на результаты будем на примере теста из предыдущего задания. Поэтому, для начала откроем его:

In [1]:
text = None
with open('war_peace_ascii_Zlobin_AS.txt', 'r', encoding='utf-8') as file:
# with open('test.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
text[:1000]

'\nCHAPTER I\n\n"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I warn you, if you don\'t tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by that\nAntichrist-I really believe he is Antichrist-I will have nothing\nmore to do with you and you are no longer my friend, no longer my\n\'faithful slave,\' as you call yourself! But how do you do? I see I\nhave frightened you-sit down and tell me all the news."\n\nIt was in July, 1805, and the speaker was the well-known Anna Pavlovna\nScherer, maid of honor and favorite of the Empress Marya Fedorovna.\nWith these words she greeted Prince Vasili Kuragin, a man of high\nrank and importance, who was the first to arrive at her reception. Anna\nPavlovna had had a cough for some days. She was, as she said, suffering\nfrom la grippe; grippe being then a new word in St. Petersburg, used\nonly by the elite.\n\nAll her invitations without exception, written in French, an

В файлах https://github.com/Andrew-Zlobin/CMDC/blob/main/task3/BWT.py и https://github.com/Andrew-Zlobin/CMDC/blob/main/task3/DC.py реализованы алгориты BWT и DC соответсвенно.

In [3]:
from BWT import BWT
from DC import DC
from utils import BWT_DC_encode_pipeline, BWT_DC_decode_pipeline, alphabet_to_number, number_to_alphabet

Напишем вспомогательные функции, чтобы убедиться, что они работают корректно:

In [4]:
def BWT_DC_encode(text):
    bwt_text = BWT.forward(text)
    alphabet = "\x01" + "".join(sorted(list(set(text))))
    int_alphabet = alphabet_to_number(alphabet)
    print("end symbol in bwt", '\x01' in bwt_text)
    print("end symbol in alphabet", '\x01' in alphabet)
    print("bwt_text = ", bwt_text)
    dc_text = DC.code(bwt_text, alphabet, BWT.get_char_spacing())
    text_len = len(bwt_text)
    array_to_encode = [text_len] + dc_text
    return array_to_encode, int_alphabet

def BWT_DC_decode(array_to_encode, int_alphabet):
    text_len = array_to_encode[0]
    dc_text = array_to_encode[1:]
    alphabet = number_to_alphabet(int_alphabet)
    dc_decoded = DC.decode(dc_text, alphabet, text_len)
    bwt_decoded = BWT.reverse(dc_decoded)
    print("end symbol in dc", '\x01' in bwt_decoded)
    return bwt_decoded

In [5]:
prepared_list, alph = BWT_DC_encode(text)

/home/dr_drew/Projects/CMDC_env/lib/python3.11/site-packages/pydivsufsort/divsufsort.py:103: UserWarning: converting str argument uses more memory
  inp_p = _get_bytes_pointer(inp)
100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3201650/3201650 [00:00<00:00, 5721997.79it/s]
IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [6]:
decoded_text = BWT_DC_decode(prepared_list, alph)

end symbol in dc False


In [7]:
decoded_text[:100]

'\nCHAPTER I\n\n"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes. But I '

Строки совпадают, значит алгоритмы работают верно

In [8]:
[i for i, j in zip(text, decoded_text) if i != j]

[]

Посмотрим на небольшую статистику массива, который получается после DC:

In [9]:
max(prepared_list[1:]), max(prepared_list)

(3007480, 3201650)

In [10]:
len(text)

3201649

In [11]:
len(prepared_list)

1378857

In [12]:
sum([1 for el in prepared_list if el <= 254])

1343183

In [13]:
sum([1 for el in prepared_list if el > 254 and el <= 65789])

35279

In [14]:
sum([1 for el in prepared_list if el > 254 and el > 65789])

395

Итого, получается, что текст длиной 3201649 символов, преобразуется в массив из 3123963 чисел, 3050004 меньше 254, 199 больше 65789 и 73522 лежат между 254 и 65789

Для сжатия этого массива будем использовать дельта-код Элиаса, арифметическое, и кодирование с переполнением
Они реалзованы соответственно в:  
https://github.com/Andrew-Zlobin/CMDC/blob/main/task3/compression/elias.py  
https://github.com/Andrew-Zlobin/CMDC/blob/main/task3/compression/arithmetic.py  
https://github.com/Andrew-Zlobin/CMDC/blob/main/task3/compression/overflow.py

И резульаты работы на тексте из предыдущего задания

### Дельта код:

Кодирование

In [28]:
%%time
!python3 compressor.py war_peace_ascii_Zlobin_AS.txt -d -f result.compressed

/home/dr_drew/Projects/CMDC_env/lib/python3.11/site-packages/pydivsufsort/divsufsort.py:103: UserWarning: converting str argument uses more memory
  inp_p = _get_bytes_pointer(inp)
100%|████████████████████████████| 3201650/3201650 [00:00<00:00, 5635599.89it/s]
CPU times: user 28.6 ms, sys: 8.34 ms, total: 36.9 ms
Wall time: 3.96 s


Размер закодированного файла:

In [29]:
!ls -l result.compressed

-rw-r--r-- 1 dr_drew dr_drew 2286674 июн 22 19:19 result.compressed


Декодирование:

In [ ]:
%%time
!python3 compressor.py result.compressed -d -f decoded.txt

CPU times: user 15.3 s, sys: 3.11 s, total: 18.4 s
Wall time: 27min 5s

Проверяем, чтобы файл совпал с исходным:

In [31]:
!cmp war_peace_ascii_Zlobin_AS.txt decoded.txt

### Арифметическое кодирование

Кодирование

In [32]:
%%time
!python3 compressor.py war_peace_ascii_Zlobin_AS.txt -a -f result.compressed

/home/dr_drew/Projects/CMDC_env/lib/python3.11/site-packages/pydivsufsort/divsufsort.py:103: UserWarning: converting str argument uses more memory
  inp_p = _get_bytes_pointer(inp)
100%|████████████████████████████| 3201650/3201650 [00:00<00:00, 5833532.83it/s]
CPU times: user 2.38 s, sys: 563 ms, total: 2.94 s
Wall time: 9min 32s


Размер закодированного файла:

In [33]:
!ls -l result.compressed

-rw-r--r-- 1 dr_drew dr_drew 2056202 июн 22 19:55 result.compressed


Декодирование:

In [34]:
%%time
!python3 compressor.py result.compressed -a -f decoded.txt

CPU times: user 2.64 s, sys: 580 ms, total: 3.22 s
Wall time: 10min 41s


Проверяем, чтобы файл совпал с исходным:

In [35]:
!cmp war_peace_ascii_Zlobin_AS.txt decoded.txt

### Кодирование с переполнением:

Кодирование

In [36]:
%%time
!python3 compressor.py war_peace_ascii_Zlobin_AS.txt -o -f result.compressed

/home/dr_drew/Projects/CMDC_env/lib/python3.11/site-packages/pydivsufsort/divsufsort.py:103: UserWarning: converting str argument uses more memory
  inp_p = _get_bytes_pointer(inp)
100%|████████████████████████████| 3201650/3201650 [00:00<00:00, 5806683.66it/s]
CPU times: user 15.5 ms, sys: 8.66 ms, total: 24.1 ms
Wall time: 3.22 s


Размер закодированного файла:

In [37]:
!ls -l result.compressed

-rw-r--r-- 1 dr_drew dr_drew 3272233 июн 22 20:06 result.compressed


Декодирование:

In [38]:
%%time
!python3 compressor.py result.compressed -o -f decoded.txt

CPU times: user 38.6 ms, sys: 8.37 ms, total: 47 ms
Wall time: 8.65 s


Проверяем, чтобы файл совпал с исходным:

In [39]:
!cmp war_peace_ascii_Zlobin_AS.txt decoded.txt

In [40]:
!ls -l war_peace_ascii_Zlobin_AS.txt

-rw-r--r-- 1 dr_drew dr_drew 3201649 июн 20 15:02 war_peace_ascii_Zlobin_AS.txt


Итого получилось, что исходный текст объёмом 3.05 мб удалось сжать до 1.95 мб арифметическим кодированием и до 2.18 мб дельта кодом. При кодировании с переполнением объём остался почти таким же, это может быть обусловлено тем, что после DC количество чисел было всего на 77686 меньше количества символов в исходном тексте, при этом если в исходном тексте один символ занимал один байт, то при сжимании массива 73522 символов кодировалось уже большим числом байт, засчёт чего объём закодированного массива увеличился. 

In [44]:
def remove_non_ascii_from_file(filename):
    with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
        content = f.read()

    ascii_content = ''.join(c for c in content if ord(c) < 128)

    with open(filename, 'w', encoding='ascii', errors='ignore') as f:
        f.write(ascii_content)

# Использование
remove_non_ascii_from_file('enwik8.txt')


In [45]:
%%time
!python3 compressor.py enwik8.txt -a -f enwik8.compressed

/home/dr_drew/Projects/CMDC_env/lib/python3.11/site-packages/pydivsufsort/divsufsort.py:103: UserWarning: converting str argument uses more memory
  inp_p = _get_bytes_pointer(inp)
100%|██████████████████████████| 99325589/99325589 [00:20<00:00, 4886308.81it/s]
^C
Traceback (most recent call last):
  File "/home/dr_drew/Projects/CMDC/task3/compressor.py", line 97, in <module>
    main()
  File "/home/dr_drew/Projects/CMDC/task3/compressor.py", line 79, in main
    encode_arithmetic(input_path, output_path)
  File "/home/dr_drew/Projects/CMDC/task3/utils.py", line 32, in handler
    res = func(text)
          ^^^^^^^^^^
  File "/home/dr_drew/Projects/CMDC/task3/utils.py", line 60, in handler
    return bytearray(num_bytes) + func(array_to_encode)
                                  ^^^^^^^^^^^^^^^^^^^^^
  File "/home/dr_drew/Projects/CMDC/task3/compressor.py", line 15, in encode_arithmetic
    return arithmetic.encode(list_to_encode)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/h

In [42]:
!ls -l enwik8.compressed

ls: невозможно получить доступ к 'enwik8.compressed': Нет такого файла или каталога


In [43]:
%%time
!python3 compressor.py enwik8.compressed -a -f enwik8_decoded.txt

Traceback (most recent call last):
  File "/home/dr_drew/Projects/CMDC/task3/compressor.py", line 97, in <module>
    main()
  File "/home/dr_drew/Projects/CMDC/task3/compressor.py", line 81, in main
    decode_arithmetic(input_path, output_path)
  File "/home/dr_drew/Projects/CMDC/task3/utils.py", line 40, in handler
    with open(input_path, 'rb') as file:
         ^^^^^^^^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'enwik8.compressed'
CPU times: user 2.08 ms, sys: 3.83 ms, total: 5.91 ms
Wall time: 188 ms


In [49]:
!python3 compressor.py test.txt -a -f test.compressed

100%|███████████████████████████████████████| 18/18 [00:00<00:00, 527954.35it/s]


In [50]:
!python3 compressor.py test.compressed -a -f decoded_test.txt

Traceback (most recent call last):
  File "/home/dr_drew/Projects/CMDC/task3/compressor.py", line 97, in <module>
    main()
  File "/home/dr_drew/Projects/CMDC/task3/compressor.py", line 81, in main
    decode_arithmetic(input_path, output_path)
  File "/home/dr_drew/Projects/CMDC/task3/utils.py", line 44, in handler
    res = func(text)
          ^^^^^^^^^^
  File "/home/dr_drew/Projects/CMDC/task3/utils.py", line 73, in handler
    bwt_decoded = BWT.reverse(dc_decoded)
                  ^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/dr_drew/Projects/CMDC/task3/BWT.py", line 77, in reverse
    return ibwt(shuffled_line)
           ^^^^^^^^^^^^^^^^^^^
  File "/home/dr_drew/Projects/CMDC/task3/BWT.py", line 34, in ibwt
    raise ValueError("EOF ('\\x01')not in text")
ValueError: EOF ('\x01')not in text
